# Euler Bernoulli Beam "solver"

The Euler-Bernoulli equation describes the relationship between the beam's deflection and the applied load

$$\frac{d^2}{dx^2}\left(EI\frac{d^2w}{dx^2}\right) = q \enspace .$$

The curve $w(x)$ describes the delection of the beam at some point $x$, $q$ is a distributed load.

This equation cannot be solve in this form in Sympy. Nevertheless, we can "trick" it to do it for us. Let us rewrite the equation as two equations

$$\begin{align}
&-\frac{d^2 M}{dx^2} = q \enspace ,\\
&- EI\frac{d^2w}{dx^2} = \frac{M}{EI} \enspace ,
\end{align}$$

where $M$ is the bending moment in the beam. We can, then, solve the two equation as if they have source terms and then couple the two solutions.

The code below do this


In [1]:
from __future__ import division
from sympy import *
from sympy import init_printing
x, E, I = symbols('x E I')
C1, C2, C3, C4 = symbols('C1 C2 C3 C4')
w, M, EI, q, f = symbols('w M EI q f', cls=Function)
init_printing()

In [2]:
M_eq = -diff(M(x), x, 2) - q(x)

M_eq

-q(x) - Derivative(M(x), x, x)

In [3]:
M_sol = dsolve(M_eq, M(x)).rhs.subs([(C1, C3), (C2, C4)])

M_sol

C3 + x*(C4 - Integral(q(x), x)) + Integral(x*q(x), x)

In [4]:
w_eq = f(x) + diff(w(x),x,2)
w_eq

f(x) + Derivative(w(x), x, x)

In [5]:
w_sol = dsolve(w_eq, w(x)).subs(f(x), M_sol/EI(x)).rhs

w_sol

C1 + x*(C2 - Integral((C3 + x*(C4 - Integral(q(x), x)) + Integral(x*q(x), x))/EI(x), x)) + Integral(x*(C3 + x*(C4 - Integral(q(x), x)) + Integral(x*q(x), x))/EI(x), x)

We want to be sure that this solution is ok. We replaced known values for $E$, $I$ and $q$ to check it.

## Cantilever beam with end load

In [6]:
sub_list = [(q(x), 0), (EI(x), E*I)]
w_sol1 = w_sol.subs(sub_list).doit()

In [7]:
L, F = symbols('L F')
# Fixed end
bc_eq1 = w_sol1.subs(x, 0)
bc_eq2 = diff(w_sol1, x).subs(x, 0)
# Free end
bc_eq3 = diff(w_sol1, x, 2).subs(x, L)
bc_eq4 = diff(w_sol1, x, 3).subs(x, L) + F/(E*I)

In [8]:
[bc_eq1, bc_eq2, bc_eq3, bc_eq4]

[C1, C2, -(C3 + C4*L)/(E*I), -C4/(E*I) + F/(E*I)]

In [9]:
constants = solve([bc_eq1, bc_eq2, bc_eq3, bc_eq4], [C1, C2, C3, C4])
constants

{C1: 0, C4: F, C2: 0, C3: -F*L}

In [10]:
w_sol1.subs(constants).simplify()

F*x**2*(3*L - x)/(6*E*I)

## Cantilever beam with uniformly distributed load

In [11]:
sub_list = [(q(x), 1), (EI(x), E*I)]
w_sol1 = w_sol.subs(sub_list).doit()

In [12]:
L = symbols('L')
# Fixed end
bc_eq1 = w_sol1.subs(x, 0)
bc_eq2 = diff(w_sol1, x).subs(x, 0)
# Free end
bc_eq3 = diff(w_sol1, x, 2).subs(x, L)
bc_eq4 = diff(w_sol1, x, 3).subs(x, L)

In [13]:
constants = solve([bc_eq1, bc_eq2, bc_eq3, bc_eq4], [C1, C2, C3, C4])

In [14]:
w_sol1.subs(constants).simplify()

x**2*(6*L**2 - 4*L*x + x**2)/(24*E*I)

## Cantilever beam with exponential loading

In [15]:
sub_list = [(q(x), exp(x)), (EI(x), E*I)]
w_sol1 = w_sol.subs(sub_list).doit()

In [16]:
L = symbols('L')
# Fixed end
bc_eq1 = w_sol1.subs(x, 0)
bc_eq2 = diff(w_sol1, x).subs(x, 0)
# Free end
bc_eq3 = diff(w_sol1, x, 2).subs(x, L)
bc_eq4 = diff(w_sol1, x, 3).subs(x, L)

In [17]:
constants = solve([bc_eq1, bc_eq2, bc_eq3, bc_eq4], [C1, C2, C3, C4])

In [18]:
w_sol1.subs(constants).simplify()

Piecewise((0, E*I == 0), ((-x**3*exp(L) - 3*x**2*(-L + 1)*exp(L) + 6*x*exp(x) - 6*x - 6*(x - 1)*exp(x) - 6)/(6*E*I), True))

## Taylor series function

In [20]:
n = symbols('n', integer=True)
cn = symbols('c_n')
q_taylor = cn*x**n

In [21]:
sub_list = [(q(x), q_taylor), (EI(x), E*I)]
w_sol1 = w_sol.subs(sub_list).doit()

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./styles/custom_barba.css', 'r').read()
    return HTML(styles)
css_styling()